In [1]:
from IPython.core.debugger import set_trace
import json
import pandas as pd
from tqdm.autonotebook import tqdm
tqdm.pandas()

import os
import sys
import re
import string
parentdir = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(parentdir)

import django
django.setup()

from data.models import Newsdata
import glob

import spacy
nlp = spacy.load('en_core_web_lg')

C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\tqdm\autonotebook\__init__.py:18: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)
C:\Users\infomax\Documents\GitHub\protobed\venv\lib\site-packages\tqdm\_tqdm.py:604: FutureWarning: The Panel class is removed from pandas. Accessing it from the top-level namespace will also be removed in the next version
  from pandas import Panel


In [97]:
def preproc(text, remove_url=True, remove_mention=True, remove_hashtag=False):
    LINEBREAK = r'\n' # str.replace에서는 r'\n'으로 검색이 안된다
    RT = '((?: rt)|(?:^rt))[^ @]?'
    EMOJI = r'[\U00010000-\U0010ffff]'
    DOTS = '…'
    LONG_BLANK = r'[ ]+'
    SPECIALS = r'([^ a-zA-Z0-9_\u3131-\u3163\uac00-\ud7a3]+)|([ㄱ-ㅣ]+)'
    PUNC = r'[{}]'.format(string.punctuation)
    
    # \u3131-\u3163\uac00-\ud7a3 는 한글을 의미함
    # URL = r'(?P<url>(https?://)?(www[.])?[^ \u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b([^ \u3131-\u3163\uac00-\ud7a3]*))'
    URL1 = r'(?:https?:\/\/)?(?:www[.])?[^ :\u3131-\u3163\uac00-\ud7a3]+[.][a-z]{2,6}\b(?:[^ \u3131-\u3163\uac00-\ud7a3]*)'
    URL2 = r'pic.twitter.com/[a-zA-Z0-9_]+'
    URL = '|'.join((URL1, URL2))
    
    HASHTAG = r'#(?P<inner_hashtag>[^ #@]+)'
    MENTION = r'@(?P<inner_mention>[^ #@]+)' 
    
    #PTNS = '|'.join((LINEBREAK, RT, URL, HASHTAG, MENTION, EMOJI))
    
    #out = {}
    #text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS)), '', text.lower())
    text = text.lower()
    
    if remove_url:
        text = re.sub(URL, ' ', text)

    if remove_mention:
        text = re.sub(MENTION, ' ', text)        
    else:
        text = re.sub(MENTION, ' \g<inner_mention>', text)
        
    if remove_hashtag:
        text = re.sub(HASHTAG, ' ', text)
    else:
        text = re.sub(HASHTAG, ' \g<inner_hashtag>', text)
        
    text = re.sub('|'.join((LINEBREAK, RT, EMOJI, DOTS, SPECIALS, PUNC)), ' ', text)
    return re.sub(LONG_BLANK, ' ', text).strip()

In [86]:
stopwords = spacy.lang.en.stop_words.STOP_WORDS; len(stopwords)

326

In [64]:
fnames = glob.glob('newsdata/downloaded/*.json'); len(fnames)

7441

In [104]:
Newsdata.objects.all().delete()

(32, {'data.Newsdata': 32})

In [105]:
texts = []
for i, fname in enumerate(tqdm(fnames[:])):
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        doc = nlp(preproc(content['text']))
        
        obj = Newsdata()
        obj.id = i + 1
        obj.pub = content['pub']
        obj.authors = ','.join(content['authors'])
        obj.title = content['title']
        obj.published_at = pd.Timestamp(content['published_at'], tz='utc')
        obj.downloaded_at = pd.Timestamp(content['downloaded_at'], tz='utc')
        obj.text = content['text']
        obj.description = content['description']
        obj.top_image = content['top_image']
        obj.url = content['url']
        obj.words = ','.join([str(noun) for noun in doc.noun_chunks if str(noun) not in stopwords])
        obj.language = content['language']
        obj.save()

In [122]:
for obj in tqdm(Newsdata.objects.all()):
    obj.words = json.dumps(obj.words.split(','))
    obj.save()

In [172]:
for obj in Newsdata.objects.filter(pub='marketwatch', url__contains='&'):
    print(obj.url)

https://marketwatch.com/story/labor-day-which-markets-are-closed-2019-08-30?mod=mw_theo_homepage&mod=mw_theo_homepage&mod=mw_theo_homepage
https://marketwatch.com/story/home-improvement-stocks-mixed-as-hurricane-expected-to-both-help-and-hurt-2019-08-30?mod=mw_theo_homepage&mod=mw_theo_homepage
https://marketwatch.com/story/here-are-2019s-biggest-stock-market-winners-and-losers-in-the-dow-sp-500-and-nasdaq-2019-08-29?mod=mw_theo_homepage&mod=mw_theo_homepage
https://marketwatch.com/story/heres-how-the-stocks-held-by-warren-buffetts-berkshire-hathaway-have-performed-in-2019-2019-08-29?mod=mw_theo_homepage&mod=mw_theo_homepage


In [187]:
Newsdata.objects.filter(pub='reuters', url__contains='?').count()

193

In [161]:
Newsdata.objects.filter(title='Is It Time to Buy Stocks Again? One Indicator Says So.')

<QuerySet [<Newsdata: [2019-09-02 | marketwatch] Is It Time to Buy Stocks Again? One Indicator Says So.>, <Newsdata: [2019-09-02 | marketwatch] Is It Time to Buy Stocks Again? One Indicator Says So.>, <Newsdata: [2019-09-02 | marketwatch] Is It Time to Buy Stocks Again? One Indicator Says So.>, <Newsdata: [2019-09-02 | wsj] Is It Time to Buy Stocks Again? One Indicator Says So.>]>

In [192]:
from tqdm.autonotebook import tqdm
tqdm.pandas()
import json
import glob
import os
import hashlib

fnames = glob.glob('newsdata/downloaded/*.json')
ext = '.json'

fnames_to_remove = []

for fname in tqdm(fnames):
    
    with open(fname, encoding='UTF-8-sig') as f:
        content = json.load(f)
        pub = content['pub']
        url = content['url']
        
        if (pub=='reuters' and '?il=0' in url) or (pub=='marketwatch' and '?mod=' in url):
            _url = clean_url(pub, url)
            hash_url = hashlib.sha1(_url.encode('utf-8')).hexdigest()
            file = os.path.join('newsdata/downloaded', hash_url + ext)
            if file in fnames:
                fnames_to_remove.append(fname)

In [168]:
def clean_url(pub, url):
    url = url.replace('http://', 'https://')
    
    if pub!='zdnet':
    # zdnet은 반드시 www가 붙어야되는 듯 (2019.08.30)
        url = url.replace('https://www.', 'https://')
    
    if pub!='thinkprogress' and url[-1]=='/':
    # thinkprogress는 뒤의 /가 반드시 필요한 듯 (2019.09.04)
        url = url[:-1]
    
    if pub=='reuters':
    # reuters는 뒤에 의미없이 ?il=0 이 붙는 경우가 허다. 무슨뜬인지는 모름 (2019.09.04)
        try: url = url[:url.index('?il=0')]
        except: pass
    
    if pub=='marketwatch':
        try: url = url[:url.index('?mod=')]
        except: pass        
    
    if pub=='wsj':
    # wsj paywall 뚫기
        try: url = url[:url.index('?mod=')]
        except: pass
        
        url += '?mod=rsswn'
        
    return url

In [190]:
[os.remove(ff) for ff in fnames_to_remove]

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,